<a href="https://colab.research.google.com/github/ione96/NLP_learning/blob/main/TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
#!pip install nlpia
#!pip install scipy
#!pip install sklearn 

In [60]:
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
from collections import Counter
import nltk
nltk.download('stopwords', quiet=True)
nltk.download('brown')
from nltk.corpus import brown
from nlpia.data.loaders import kite_text, kite_history
from nlpia.data.loaders import harry_docs 
from collections import OrderedDict
import copy

sentence = """The faster Harry got to the store, the faster Harry, the faster, would get home."""
tokens = tokenizer.tokenize(sentence.lower())
bag_of_words = Counter(tokens)
print(bag_of_words.most_common(4))  # 4 самых частых терма

times_harry_appers = bag_of_words['harry']
num_unique_words = len(bag_of_words)
tf = times_harry_appers / num_unique_words
print(round(tf, 4)) # частотность терма harry

stopwords = nltk.corpus.stopwords.words('english')
tokens = tokenizer.tokenize(kite_text.lower())
tokens = [x for x in tokens if x not in stopwords]
kite_counts = Counter(tokens)
print(kite_counts) # частотность термов без стов-слов

document_vector = []
doc_length = len(tokens)
for key, value in kite_counts.most_common(): document_vector.append(value / doc_length)
print(document_vector) # вектор 

doc_tokens = []
for doc in harry_docs:  doc_tokens += [sorted(tokenizer.tokenize(doc.lower()))]
all_doc_tokens = sum(doc_tokens, [])
lexicon = sorted(set(all_doc_tokens))
print(len(lexicon)) # количество токенов 
print(lexicon) # слова из лексикона

zero_vector = OrderedDict((token, 0) for token in lexicon) 
doc_vectors = []
for doc in harry_docs:
  vec = copy.copy(zero_vector)
  tokens = tokenizer.tokenize(doc.lower())
  token_counts = Counter(tokens)
  for key, value in token_counts.items():
    vec[key] = value / len(lexicon)
  doc_vectors.append(vec)
print(doc_vectors) # Встречаемость каждого токена в лексиконе

import math
def cosine_sim(vec1, vec2): # вычисление косинусного сходства
  """ Let's convert our dictionaries to lists for easier matching."""
  vec1 = [val for val in vec1.values()]
  vec2 = [val for val in vec2.values()]

  dot_prod = 0
  for i, v in enumerate(vec1):
    dot_prod += v * vec2[i]

  mag_1 = math.sqrt(sum([x**2 for x in vec1]))
  mag_2 = math.sqrt(sum([x**2 for x in vec2]))

  return dot_prod / (mag_1 * mag_2)

#puncs = set((',', '.', '--', '-', '!', '?', ':', ';', '``', "''", '(', ')', '[', ']'))
#word_list = (x.lower() for x in brown.words() if x not in puncs)
#token_counts = Counter(word_list)
#print(token_counts.most_common(20)) # 20 наиболее встречающихся токенов (закон Ципфа)

kite_intro = kite_text.lower()
intro_tokens = tokenizer.tokenize(kite_intro)
kite_history = kite_history.lower()
history_tokens = tokenizer.tokenize(kite_history)
intro_total = len(intro_tokens)
history_total = len(history_tokens)

intro_tf = {}
history_tf = {}
intro_counts = Counter(intro_tokens)
intro_tf['kite'] = intro_counts['kite'] / intro_total
history_counts = Counter(history_tokens)
history_tf['kite'] = history_counts['kite'] / history_total
intro_tf['and'] = intro_counts['and'] / intro_total
history_tf['and'] = history_counts['and'] / history_total
intro_tf['china'] = intro_counts['china'] / intro_total
history_tf['china'] = history_counts['china'] / history_total
print(intro_tf)  #встречаемость kite, and, china в двух документах
print(history_tf)

num_docs_containing_and = 0
for doc in [intro_tokens, history_tokens]:
  if 'and' in doc:
    num_docs_containing_and +=1
num_docs_containing_kite = 0
for doc in [intro_tokens, history_tokens]:
  if 'kite' in doc:
    num_docs_containing_kite +=1
num_docs_containing_china = 0
for doc in [intro_tokens, history_tokens]:
  if 'china' in doc:
    num_docs_containing_china +=1
num_docs = 2
intro_idf = {}
history_idf = {}
intro_idf['and'] = num_docs / num_docs_containing_and
history_idf['and'] = num_docs / num_docs_containing_and
intro_idf['kite'] = num_docs / num_docs_containing_kite
history_idf['kite'] = num_docs / num_docs_containing_kite
intro_idf['china'] = num_docs / num_docs_containing_china
history_idf['china'] = num_docs / num_docs_containing_china
print(intro_idf)
print(history_idf) # idf - общее кол-во документов / кол-во документов в котором встречается слово

intro_tfidf = {}
intro_tfidf['and'] = intro_tf['and'] * intro_idf['and']
intro_tfidf['kite'] = intro_tf['kite'] * intro_idf['kite']
intro_tfidf['china'] = intro_tf['china'] * intro_idf['china']
history_tfidf = {}
history_tfidf['and'] = history_tf['and'] * history_idf['and']
history_tfidf['kite'] = history_tf['kite'] * history_idf['kite']
history_tfidf['china'] = history_tf['china'] * history_idf['china']
print(intro_tfidf)
print(history_tfidf) #TF-IDF

# вычисление в логорифмическом пространстве:
#log_tf = log(term_occurences_in_doc) - log(num_terms_in_doc)
#log_log_idf = log(log(total_num_docs) - log(num_docs_containing_term))
#log_tf_idf = log_tf + log_idf

document_tfidf_vectors = []
for doc in harry_docs:
  vec = copy.copy(zero_vector)
  tokens = tokenizer.tokenize(doc.lower())
  token_counts = Counter(tokens)
  for key, value in token_counts.items():
    docs_containing_key = 0
    for _doc in harry_docs:
      if key in _doc:
        docs_containing_key += 1
    tf = value / len(lexicon)
    if docs_containing_key:
      idf = len(harry_docs) / docs_containing_key
    else:
      idf = 0
    vec[key] = tf * idf 
  document_tfidf_vectors.append(vec)
print(document_tfidf_vectors) # TF-IDF для предложений о harry

query = "How long does it take to get to the store?"
query_vec = copy.copy(zero_vector)
tokens = tokenizer.tokenize(query.lower())
token_counts = Counter(tokens)
for key, value in token_counts.items():
  docs_containing_key = 0
  for _doc in harry_docs:
    if key in _doc.lower():
      docs_containing_key +=1
  if docs_containing_key == 0:
    continue
    tf = value / len(tokens)
    idf = len(harry_docs) / docs_containing_key
  query_vec[key] = tf * idf
print(cosine_sim(query_vec, document_tfidf_vectors[0])) # косинустные соответствия для запроса и предложений
print(cosine_sim(query_vec, document_tfidf_vectors[1]))
print(cosine_sim(query_vec, document_tfidf_vectors[2]))



[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[('the', 4), ('faster', 3), (',', 3), ('harry', 2)]
0.1818
Counter({'kite': 16, ',': 15, 'kites': 8, 'wing': 5, 'lift': 4, 'may': 4, 'also': 3, 'kiting': 3, 'flown': 3, 'tethered': 2, 'craft': 2, 'air': 2, 'consists': 2, 'tethers': 2, 'anchors.': 2, 'often': 2, 'bridle': 2, 'wind': 2, "'s": 2, 'designed': 2, ';': 2, 'moving': 2, 'technical': 2, 'even': 2, 'called': 2, 'surface': 2, 'pressure': 2, 'drag': 2, 'force': 2, '.': 2, 'used': 2, 'power': 2, 'traditionally': 1, 'heavier-than-air': 1, 'surfaces': 1, 'react': 1, 'create': 1, 'drag.': 1, 'wings': 1, 'guide': 1, 'face': 1, 'correct': 1, 'angle': 1, 'it.': 1, 'needed': 1, 'sailplane': 1, 'launch': 1, 'tether': 1, 'meets': 1, 'single': 1, 'point.': 1, 'fixed': 1, 'untraditionally': 1, 'tether-set-coupled': 1, 'sets': 1, 'though': 1, 'system': 1, 'still': 1, 'kite.': 1, 'sustains': 1, 'flight': 1, 'generated': 1, 'flows': 1, 